In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve,convolve2d
import math
from PIL import Image
import numpy as np 
import copy
from collections import deque

In [2]:
def image_registration(img1, img2):
    img1_color = cv2.imread(img1)
    img2_color = cv2.imread(img2)
    img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
    height, width = img2.shape
    orb_detector = cv2.ORB_create(5000)
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    matches = matcher.match(d1, d2)
    matches.sorted(key = lambda x: x.distance)
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
      p1[i, :] = kp1[matches[i].queryIdx].pt
      p2[i, :] = kp2[matches[i].trainIdx].pt
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)
    transformed_img = cv2.warpPerspective(img1_color,homography, (width, height))
    plt.axis('off')
    plt.imshow(transformed_img,cmap='gray', vmin=0, vmax=255)
    plt.show()

In [3]:
def motionTrackingFunction(img1,img2):
    Iref=img1
    Inext=img2
    Iref=np.array(Iref).astype(np.float32)
    Inext=np.array(Inext).astype(np.float32)
    kernel_x = np.array([[-1., 1.], [-1., 1.]])*.25
    kernel_y = np.array([[-1., -1.], [1., 1.]])*.25
    kernel_t = np.array([[1., 1.], [1., 1.]])*.25
    Iref = Iref / 255. 
    Inext = Inext / 255. 
    Ix=cv2.filter2D(Iref,-1,kernel=kernel_x)
    Iy=cv2.filter2D(Iref,-1,kernel=kernel_y)
    It=cv2.filter2D(Iref,-1,kernel=kernel_t)+cv2.filter2D(Inext,-1,kernel=kernel_x)
    Ix,Iy,It=np.array(Ix),np.array(Iy),np.array(It)
    u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
    return u

for i in range(0,240,30):
    Iref=cv2.imread(r'D:\GSU\CV\Assignment3_Harshith\Qusetion2\frammed\frame%d.jpg'%i,cv2.IMREAD_GRAYSCALE)
    Inext=cv2.imread(r'D:\GSU\CV\Assignment3_Harshith\Qusetion2\frammed\frame%d.jpg'%(i+30),cv2.IMREAD_GRAYSCALE)
    print("Motion function estamite for frame" + str(i) + " frame" + str(i+30) + str(motionTrackingFunction(Iref,Inext)))

C:\Users\Harsh\AppData\Local\Temp\ipykernel_13452\2992858355.py:15: RuntimeWarning: divide by zero encountered in divide
  u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))
C:\Users\Harsh\AppData\Local\Temp\ipykernel_13452\2992858355.py:15: RuntimeWarning: invalid value encountered in divide
  u=np.divide(It,np.sqrt(np.square(Ix)+np.square(Iy)))


Motion function estamite for frame0 frame30[[      inf       inf  91.00001 ...       inf       inf       inf]
 [      inf       inf  91.00001 ...       inf       inf       inf]
 [129.40056 129.40056 127.98634 ...       inf       inf       inf]
 ...
 [      inf       inf       inf ...       inf       inf       inf]
 [      inf       inf       inf ...       inf       inf       inf]
 [      inf       inf       inf ...       inf       inf       inf]]
Motion function estamite for frame30 frame60[[       inf        inf        inf ...        inf        inf        inf]
 [       inf        inf        inf ...        inf        inf        inf]
 [       inf        inf        inf ...        inf        inf        inf]
 ...
 [ 85.55993   85.55993   86.974144 ... 282.13562  201.5      202.00002 ]
 [ 86.97422   86.974144        inf ...        inf        inf        inf]
 [       inf        inf        inf ...        inf        inf        inf]]
Motion function estamite for frame60 frame90[[      inf      